In [1]:
import pandas as pd
import numpy as np
import os
import sys
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Reproducible results
random_state = 42
np.random.seed(random_state)

# Data

In [3]:
def get_data(one_hot_enc=True, reshape=False, train_val_test=False):
    DATA_PATH = "../data/"
    file_list = os.listdir(DATA_PATH)
    images_path = file_list[0]
    labels_path = file_list[1]
    images_path_full = os.path.join(DATA_PATH + images_path)
    labels_path_full = os.path.join(DATA_PATH + labels_path)
    X = pd.read_csv(images_path_full)
    if reshape:
        X = X.values.astype('float32')
        # Normalize data
        X = X / 255
        # Reshape for cnn 
        X = X.reshape([-1, 28, 28, 1]).astype('float32')
    else:
        # Normalize data
        X = X / 255
    y = pd.read_csv(labels_path_full)
    # Encode labels
    if one_hot_enc:
        encoder = OneHotEncoder(sparse=False, categories='auto')
        y = encoder.fit_transform(y)
    # Divide into train and test set 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
    if train_val_test:
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=random_state)
        return X_train, X_val, X_test, y_train, y_val, y_test
    else:
        return X_train, X_test, y_train, y_test

In [4]:
def print_data_shapes_val(X_train, X_val, X_test, y_train, y_val, y_test):
    print("X_train_nn: ", X_train.shape)
    print("X_val_nn: ", X_val.shape)
    print("X_test_nn: ", X_test.shape)
    print("y_train_nn: ", y_train.shape)
    print("y_val_nn: ", y_val.shape)
    print("y_test_nn: ", y_test.shape)

In [5]:
def print_data_shapes_nval(X_train, X_test, y_train, y_test):
    print("X_train_nn: ", X_train.shape)
    print("X_test_nn: ", X_test.shape)
    print("y_train_nn: ", y_train.shape)
    print("y_test_nn: ", y_test.shape)

# Classifiers

### Feed Forward Neural Net

In [6]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras import regularizers

C:\Users\delsin\AppData\Local\Continuum\anaconda3\envs\dat158\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [7]:
X_train_nn, X_val_nn, X_test_nn, y_train_nn, y_val_nn, y_test_nn = get_data(train_val_test=True)

In [8]:
# Params

# Model compile
adam = Adam(lr=1e-3)
metrics = ['accuracy']
loss = 'categorical_crossentropy'

# Model architecture
regularizer_l2 = regularizers.l2(0.01)

# Model fit
epochs = 12
batch_size = 64

In [9]:
print_data_shapes_val(X_train_nn, X_val_nn, X_test_nn, y_train_nn, y_val_nn, y_test_nn)

X_train_nn:  (39199, 784)
X_val_nn:  (16800, 784)
X_test_nn:  (14000, 784)
y_train_nn:  (39199, 10)
y_val_nn:  (16800, 10)
y_test_nn:  (14000, 10)


In [10]:
model = Sequential()
model.add(Dense(256, input_dim=784, activation='relu', kernel_regularizer=regularizer_l2))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu', kernel_regularizer=regularizer_l2))
model.add(Dense(10, activation='softmax'))

In [11]:
model.compile(loss=loss, optimizer=adam, metrics=metrics)

In [12]:
start_nn = time.time()
model.fit(X_train_nn, y_train_nn, validation_data=(X_val_nn, y_val_nn), epochs=epochs, batch_size=batch_size)
end_nn = time.time()
scores = model.evaluate(X_test_nn, y_test_nn)

Train on 39199 samples, validate on 16800 samples
Epoch 1/12
39199/39199 [==============================] - 3s 77us/step - loss: 1.0281 - acc: 0.8396 - val_loss: 0.5649 - val_acc: 0.9104
Epoch 2/12
39199/39199 [==============================] - 3s 69us/step - loss: 0.4634 - acc: 0.9327 - val_loss: 0.4888 - val_acc: 0.9145
Epoch 3/12
39199/39199 [==============================] - 2s 63us/step - loss: 0.3810 - acc: 0.9469 - val_loss: 0.3731 - val_acc: 0.9454
Epoch 4/12
39199/39199 [==============================] - 3s 65us/step - loss: 0.3401 - acc: 0.9502 - val_loss: 0.3545 - val_acc: 0.9470
Epoch 5/12
39199/39199 [==============================] - 3s 64us/step - loss: 0.3158 - acc: 0.9533 - val_loss: 0.3380 - val_acc: 0.9470
Epoch 6/12
39199/39199 [==============================] - 3s 65us/step - loss: 0.2808 - acc: 0.9580 - val_loss: 0.3180 - val_acc: 0.9463
Epoch 7/12
39199/39199 [==============================] - 3s 66us/step - loss: 0.2703 - acc: 0.9594 - val_loss: 0.2996 - val_acc

In [13]:
print("Scores: ", scores, "\n Training time: %.2f" % end_nn, " seconds")

Scores:  [0.24156733674662453, 0.9622142857142857] 
 Training time: 1540287525.74  seconds


> Can see that the model is somewhat overfitting - Will try to implement *Regularizer*
<br>
This resulted in a loss in accuracy, but there is a trade off between accuracy and overfitting

### CNN

In [14]:
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.datasets import mnist
from keras.utils import np_utils

In [15]:
X_train_cnn, X_val_cnn, X_test_cnn, y_train_cnn, y_val_cnn, y_test_cnn = get_data(one_hot_enc=False, reshape=True, train_val_test=True)

In [16]:
y_train_cnn = np_utils.to_categorical(y_train_cnn)
y_val_cnn = np_utils.to_categorical(y_val_cnn)
y_test_cnn = np_utils.to_categorical(y_test_cnn)

In [17]:
print_data_shapes_val(X_train_cnn, X_val_cnn, X_test_cnn, y_train_cnn, y_val_cnn, y_test_cnn)

X_train_nn:  (39199, 28, 28, 1)
X_val_nn:  (16800, 28, 28, 1)
X_test_nn:  (14000, 28, 28, 1)
y_train_nn:  (39199, 10)
y_val_nn:  (16800, 10)
y_test_nn:  (14000, 10)


In [18]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
start_cnn = time.time()
model.fit(X_train_cnn, y_train_cnn, validation_data=(X_val_cnn, y_val_cnn), epochs=4, batch_size=200)
end_cnn = time.time()
scores = model.evaluate(X_test_cnn, y_test_cnn)

Train on 39199 samples, validate on 16800 samples
Epoch 1/4
39199/39199 [==============================] - 16s 407us/step - loss: 0.2035 - acc: 0.9373 - val_loss: 0.0751 - val_acc: 0.9764
Epoch 2/4
39199/39199 [==============================] - 16s 414us/step - loss: 0.0624 - acc: 0.9803 - val_loss: 0.0598 - val_acc: 0.9818
Epoch 3/4
39199/39199 [==============================] - 16s 417us/step - loss: 0.0437 - acc: 0.9858 - val_loss: 0.0519 - val_acc: 0.9843
Epoch 4/4
14000/14000 [==============================] - 2s 143us/step


In [ ]:
print("Scores: ", scores, "\n Training time: %.2f" % end_cnn, " seconds")

Scores:  [0.04617324691006382, 0.9867142857142858] 
 Training time: 1540287597.83  seconds


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
X_train_rf, X_test_rf, y_train_rf, y_test_rf = get_data()

In [ ]:
print_data_shapes_nval(X_train_rf, X_test_rf, y_train_rf, y_test_rf)

X_train_nn:  (55999, 784)
X_test_nn:  (14000, 784)
y_train_nn:  (55999, 10)
y_test_nn:  (14000, 10)


In [ ]:
random_f_clf = RandomForestClassifier(n_estimators=50,
                                     n_jobs = 2,
                                     random_state=random_state)
start_rf = time.time()
random_f_clf.fit(X_train_rf, y_train_rf)
end_rf = time.time()
y_pred_rf = random_f_clf.predict(X_test_rf)
scores = accuracy_score(y_test_rf, y_pred_rf)

In [ ]:
print("Scores: ", scores, "\n Training time: %.2f" % end_rf, " seconds")

Scores:  0.8957857142857143 
 Training time: 1540287621.94  seconds


### XGBoost

In [ ]:
# If needing to install xgboost (using conda)
# ! conda install py-xgboost

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
X_train_xg, X_test_xg, y_train_xg, y_test_xg = get_data(one_hot_enc=False)

In [ ]:
print_data_shapes_nval(X_train_xg, X_test_xg, y_train_xg, y_test_xg)

X_train_nn:  (55999, 784)
X_test_nn:  (14000, 784)
y_train_nn:  (55999, 1)
y_test_nn:  (14000, 1)


In [ ]:
params = {
        'gamma': [0.5, 2],
        'subsample': [0.6, 1.0],
        'colsample_bytree': [0.6, 1.0],
        'max_depth': [3, 5],
        'n_estimators': [5, 10]
        }

Fun to try with more parameters, but my poor CPU cannot take any more..

## Disclaimer: This takes a lot of time

In [ ]:
%%capture 
xgb_clf = XGBClassifier()
rs = GridSearchCV(xgb_clf,
                  params,
                  cv=2,
                  scoring="accuracy",
                  n_jobs=1,
                  verbose=2)
start_xgb = time.time()
rs.fit(X_train_xg, y_train_xg.values.ravel())
end_xgb = time.time()

In [ ]:
optim_est_xg = rs.best_estimator_
print(optim_est_xg)

In [ ]:
y_pred_xg = optim_est_xg.predict(X_test_xg)
scores = accuracy_score(y_test_xg, y_pred_xg)

In [ ]:
print("Scores: ", scores, "\n Training time: %.2f" % end_xgb, " seconds")